In [14]:
## only do it the first time (by removing the #)
#import sys
#!{sys.executable} -m pip install plotly
#!{sys.executable} -m pip install cvxpy

In [15]:
import plotly.graph_objects as go 
from plotly import offline
import numpy as np
import cvxpy as cp
from time import time
from copy import copy, deepcopy
import pandas as pd
COLORS = {"school" : "green", "warehouse": "blue", "central":"red","road":"grey"}
SIZES = {"school" : 30, "warehouse": 40,"central": 50}
TITLE = 'WFP Inventory problem'

# Defining framework 

Here we define the framework for visualization, then another function will solve the problem, and give us the decision variables $x_{mk}^t$ (how much food is delivered to school $m$ during tour $k$ at time $t$) and $y_k^t$ (boolean to know if the tour $k$ should be done at time $t$)

In [16]:
class School :
    def __init__(self, position, capacity,consumption,name):
        self.name = name
        self.pos = position
        self.C = capacity
        self.inventory = capacity
        self.q = consumption

    def eat(self):
        self.inventory -= self.q
        
    def receive(self,x):
        self.inventory += x
        
class Warehouse : 
    def __init__(self, position, capacity,name):
        self.name = name
        self.pos = position
        self.C = capacity
        self.inventory = capacity
        self.pickup_cost = 0

    def deliver(self, quantity):
        self.inventory -= quantity
        
    def receive(self,quantity):
        self.inventory += quantity
        
    def compute_pickup_cost(self,central):
        self.pickup_cost = round( 2*np.sqrt( (self.pos[0]-central[0])**2 + (self.pos[1]-central[1])**2) )
    
    def make_arrow(self,central):
        
        x1 = central[0]
        y1 = central[1]
        x2 = self.pos[0]
        y2 = self.pos[1]
        
        text = "cost = " + str(round(self.pickup_cost,2))

        return go.Scatter(x=[(7*x1+x2)/8,(x1+7*x2)/8],
                y=[(7*y1+y2)/8,(y1+7*y2)/8],
                line= dict(color=COLORS["road"]),
                text = [text,text],
                hoverinfo='text',
                visible = False
                    )

class Route : 
    def __init__(self, warehouse, SCHOOLS):
        self.w = warehouse
        self.S = SCHOOLS
        self.Mk = len(self.S)

    def set_Q(self,Q):
        self.Q = Q      

    def compute_edges(self):
        '''
        Make a list of edges for the route, that is, for each edge, a couple of objects Warehouse or School
        '''
        self.edges = [(self.w,self.S[0])]
        for i in range(self.Mk-1):
            self.edges.append((self.S[i],self.S[i+1]))
        self.edges.append((self.S[-1],self.w))

    def cost(edge,D):
        return D.loc[edge[0].name, edge[1].name]

    def compute_costs(self,D,fixed_costs=0.):
        '''
        compute the cost of a tour with respect to the distances and the fixed cost
        '''
        self.costs = []
        for e in self.edges :
            self.costs.append(Route.cost(e,D))
        
        self.cost = sum(self.costs) + fixed_costs

    def compute_C(self):
        self.room_available = [s.C-s.inventory for s in self.S]
        self.C = min(self.Q, sum(self.room_available))
                
    def do_tour(self):
        '''
        Change the inventory with respect to a tour and its X
        '''
        self.w.deliver(sum(self.X))
        for i in range(len(self.X)):
            self.S[i].receive(self.X[i])

    def make_arrows(self) : 
        self.arrows= []
        for e in range(self.Mk+1):
            edge = self.edges[e]
            x1, x2, y1, y2 = edge[0].pos[0], edge[1].pos[0], edge[0].pos[1], edge[1].pos[1]
            text = "cost = "+ str(round(self.costs[e],1))

            arrow = go.Scatter(x=[(7*x1+x2)/8,(x1+7*x2)/8],
                    y=[(7*y1+y2)/8,(y1+7*y2)/8],
                    line= dict(color=COLORS["road"]),
                    text = [text,text],
                    hoverinfo='text',
                    visible = False
                    )

            self.arrows.append(arrow)
            
    def set_no(self,indices):
        names = [s.name for s in self.S]
        self.numbers_RtoS = np.array([indices[name] for name in names  ])


In [17]:
class Map :
    def __init__(self, central, schools, warehouses, possible_routes = [], Q = 5., D=None, Q2 = 10.):
        '''
        This object represents the state of the problem at some time t 
        '''
        
        self.central = central
        self.S = schools
        self.W = warehouses
        self.R_possible = possible_routes
        self.Q = Q
        self.Q2= Q2
        self.D = D
        self.M, self.N, self.K = len(schools),len(warehouses), len(possible_routes )
        self.warehouse_to_deliver = []
        self.compute_pickup_costs()
        
        
        find_index_s =  {schools[i].name : i for i in range(self.M)}
        for r in self.R_possible : 
            r.set_Q(Q)
            r.set_no(find_index_s)
            
        if D is None :
            positions = np.array([w.pos for w in warehouses] + [s.pos for s in schools]) 
            d = Map.compute_distances(positions)
            names = [w.name for w in warehouses]+[s.name for s in schools]
            self.D = pd.DataFrame(data=d, columns=names, index=names)
            
        else: 
            self.D = D

        
        self.t = 0.
        self.total_cost = 0
        self.cost = 0
        
        for k in range(self.K): 
            self.R_possible[k].number = k

    def build_Rmat(self):
        '''
        Method that build the numpy array Rmat such that r[k,m] = 1 if & only if S[m] in R[k]
        '''
        self.Rmat = np.zeros((self.K,self.M),dtype = bool)
        names = [s.name for s in self.S]
        for k in range(self.K):
            r = self.R_possible[k]
            for stop in r.S : 
                m = names.index(stop.name)
                self.Rmat[k,m] = True

    def compute_pickup_costs(self):
        for w in self.W :
            w.compute_pickup_cost(self.central)
    
    def compute_distances(positions):
        return np.sqrt(np.sum((positions-positions[:,np.newaxis])**2,axis=2))
                            
    def compute_central_deliveries(self):
        for j in range(self.N):
            if self.W[j].inventory < 0 : 
                self.warehouse_to_deliver.append(j)
                self.W[j].receive(self.Q2)
    
    def compute_edges(self):
        for r in self.R_possible : 
            r.compute_edges()

    def compute_costs(self, fixed_costs=0.):
        for r in self.R_possible : 
            r.compute_costs(self.D, fixed_costs=fixed_costs)

    def select_tours(self,y):
        # y should be a boolean vector of length K here that state if the tour k should be done
        self.R = np.arange(self.K)[y]
        
    def compute_X(self,x):
        # x is a matrix KxM that gives the quantity of food to be delivered to each school for each tour
        for k in self.R : 
            r = self.R_possible[k]
            r.X = x[k,r.numbers_RtoS]
                        
    def do_tours(self):
        for k in self.R :
            r = self.R_possible[k] 
            r.do_tour()
            self.cost += r.cost
        self.compute_central_deliveries()
        for j in self.warehouse_to_deliver : 
            self.cost+=self.W[j].pickup_cost
        
        
        self.total_cost += self.cost
        self.title = TITLE + "        Cost = %s          Total Cost = " %str(round(self.cost)) + str(round(self.total_cost))
        self.title = self.title + "   Truck 1 capacity : "+ str(self.Q) + "   Truck 2 capacity : "+ str(self.Q2)
        
        self.cost = 0

    def eat(self):
        for s in self.S:
            s.eat()

    def init_draw(self):

        # create arrows
        for k in range(self.K): 
            r = self.R_possible[k].make_arrows()

        #plot the schools
        plot_schools = go.Scatter(x=[school.pos[0] for school in self.S],
                        y=[school.pos[1] for school in self.S],
                        mode='markers',
                        name='schools',
                        marker=dict(symbol='circle-dot',
                                        size=SIZES["school"],
                                        color=COLORS["school"]
                                        ),
                        text=[s.name+" C = "+str(s.C)+"  ;   q = "+str(s.q) for s in self.S],
                        hoverinfo='text',
                        opacity=0.8
                        )
        #plot the warehouses
        plot_warehouses = go.Scatter(x=[warehouse.pos[0] for warehouse in self.W],
                            y=[warehouse.pos[1] for warehouse in self.W],
                            mode='markers',
                            name='warehouses',
                            marker=dict(symbol='circle-dot',
                                            size=SIZES["warehouse"],
                                            color=COLORS["warehouse"]
                                            ),
                            text=[w.name+" C = "+str(w.C) for w in self.W],
                            hoverinfo='text',
                            opacity=0.8
                            )
        
        plot_central = go.Scatter(x=[self.central[0]],
                            y=[self.central[1]],
                            mode='markers',
                            name='central',
                            marker=dict(symbol='circle-dot',
                                            size=SIZES["central"],
                                            color=COLORS["central"]
                                            ),
                            text=["Central warehouse"],
                            hoverinfo='text',
                            opacity=0.8
                            )

        
        axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=True,
            showgrid=True,
            showticklabels=True,
            )

        self.title = TITLE + "        Cost = %s          Total Cost = " %str(self.cost) + str(self.total_cost)
        self.title = self.title + "   Truck 1 capacity : "+ str(self.Q) + "   Truck 2 capacity : "+ str(self.Q2)
        
        
        tuples = [(school.pos, "I = "+str(school.inventory),'black') for school in self.S] 
        tuples = tuples + [(warehouse.pos, "I = "+str(warehouse.inventory),'yellow') for warehouse in self.W]
        tuples.append((self.central,"CENTRAL", 'black') )
        
        
        layout = dict(
            title= self.title,
              annotations= Map.make_annotations(tuples), 
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)',
              updatemenus= []
        )

        self.layout = layout
        self.data = [plot_schools,plot_warehouses, plot_central]
        self.arrows = [-1,-1,-1]
        for r in self.R_possible : 
            number = r.number
            for x in r.arrows : 
                self.data.append(x)
                self.arrows.append(number)
        
        for w in self.W :
            pick_up = w.make_arrow(self.central)
            self.data.append(pick_up)
            self.arrows.append(w.name)
       
    def make_annotations(tuples):
        annotations = []
        for (pos,txt,color) in tuples:
            annotations.append(
                dict(
                    text=txt, # text within the circles
                    x=pos[0], y=pos[1],
                    xref='x1', yref='y1',
                    font=dict(color=color, size=15),
                    showarrow=False)
            )
        return annotations

    def make_updatemenu(self):
        tuples = [(school.pos, "I = "+str(round(school.inventory,2)),'black') for school in self.S] + [(warehouse.pos, "I = "+str(round(warehouse.inventory,2)),'yellow') for warehouse in self.W]
        tuples.append( (self.central,"CENTRAL", 'black') )
        
        l = len(self.data)
        visible = [True]*3 + [False]*(l-3)

        if self.t.is_integer():
            period = " (before lunch)"
            for k in self.R : 
                # decide which arrows are visible, 
                r = self.R_possible[k]
                for i in range(r.Mk):
                    e = r.edges[i]
                    x = (e[0].pos[0]+e[1].pos[0]) / 2 + 1.
                    y = (e[0].pos[1]+e[1].pos[1]) / 2 + 1.
                    tuples.append(([x,y], str(round(r.X[i],2))+ "  ", COLORS["road"]))

                i = self.arrows.index(r.number)
                a = len(r.arrows)
                visible[i:i+a]=[True]*a
                
                
            for j in self.warehouse_to_deliver : 
                visible[l-self.N+j ] = True
        
        else : 
            period = "  (evening)"
            visible[3:] = [False]*(l-3)


        annotations = Map.make_annotations(tuples)

        return dict(label="t = "+str(int(self.t))+ period, method = "update", args=[{"visible" : copy(visible)  },{"annotations": annotations, "title":self.title }])

    def run(self,solver,T=10, H=4):
        '''
        final function that make the process continue for T days, and plot it into self.fig 
        Also have to put as an input a function f that will build X and Y
        '''
        self.build_Rmat()
        self.compute_edges()
        self.compute_costs()
        
        t0 = time()
        X,Y = solver(cost=np.array([r.cost for r in self.R_possible]),
                q      =np.array([s.q for s in self.S]),
                C      =np.array([s.C for s in self.S]),
                I_init = np.array([s.inventory for s in self.S]),
                r      =self.Rmat,
                Q=self.Q,T=T,H=H)
        
        print("Running time for solver is %f sec" %(time()-t0))
        self.init_draw()
        L1,L2 = [],[]
        for i in range(T):
            #morning
            self.select_tours(Y[i])
            self.compute_X(X[i])
            self.do_tours()
            L1.append(self.make_updatemenu())
            self.R = []
            self.warehouse_to_deliver = []
            self.t += 0.5
            
            # evening
            self.eat()
            L2.append(self.make_updatemenu())
            self.t += 0.5



        self.layout["updatemenus"]    = [ dict(buttons = L1, direction = "up",x=0.,y=0.),dict(buttons = L2, direction = "up",x=0.3,y=0.) ]
        

        self.fig = dict(data=self.data, layout=self.layout)


# Defining solver

\begin{eqnarray}
&\min& \sum_t \sum_k c_k y_k^t \\
& & \sum_m x_{km}^t \leq Q y_k^t \qquad \forall t, k \\
& & x_{km}^t \leq Q r_{km} \qquad \forall t,m, k\\
& & q_m  \leq  I_m^0 + \sum_{k} \sum_{s\leq t}  x_{km}^s - t q_m \leq C_m \qquad \forall t,m \\
\end{eqnarray}

In [18]:
def aux1(cost,q,C,r,I_init,Q,T,H=None):
    M,K = len(q),len(cost)
    
    Y = cp.Variable((T,K),boolean=True)
    
    X = {}
    constraints = []
    I = I_init
    for t in range(T):
        X[t] = cp.Variable((K,M),nonneg=True)
        
        constraints.append(cp.sum(X[t],axis=1)<=Q*Y[t] )
        constraints.append(X[t]<=Q*r)
        
        I = I + cp.sum(X[t],axis=0) - q
        constraints.append( I <= C - q )
        constraints.append( I >= 0 )
        
    
    problem = cp.Problem(cp.Minimize(cp.sum(Y@cost)), constraints)
    problem.solve()
    
    return np.array([X[t].value for t in range(T)]),np.array(Y.value,dtype=bool)

def solver1(cost,q,C,r,I_init,Q,T,H=None):
    if H is None : 
        return aux1(cost,q,C,r,I_init,Q,T,H=None)
    
    else : 
        X_weekly, Y_weekly = [],[]
        K,M = len(cost),len(I_init)
        R_quantity = np.ones(K)
        R_nbr = K
        R_selected = np.array([True]*K)
        t = 0
        I = I_init[:]
        while t<T : 
            X,Y = np.zeros((H,K,M)), np.zeros((H,K),dtype=bool)
            X[:,R_selected,:],Y[:,R_selected] = aux1(cost[R_selected],q,C,r[R_selected,:],I,Q,H)
            X_weekly.append(X),Y_weekly.append(Y)
            
            t = t + H
            I = I - H*q + X.sum(axis=0).sum(axis=0)
            
            R_done = Y.sum(axis=0)
            R_quantity = R_quantity + R_done
            R_nbr += R_done.sum()
            R_selected = (sum(R_selected)*R_quantity >= R_nbr*0.1)
            
            
            
        return np.concatenate(X_weekly,axis=0), np.concatenate(Y_weekly,axis=0)
              


Which can also be writen as follow : with $L$ the lower triangular $T\times T$ matrix with only ones and $u = 1_T$.

$X_m$ some matrices of size $T\times K$, $Y$ is a matrix of size $T \times K$

Notation : $\circ$ when we repeat the vector $T$ time to change it to a matrix.

Let $a$ be the vector $(1,2,3,....,T)$

\begin{eqnarray}
&\min&  sum( Y c) \\
& & \sum_m x_{m} \leq Y \\
& & x_m \leq r_{:,m} \circ \qquad \forall m\\
& & \frac{q_m}{Q}\circ  \leq  \frac{I_m^0}{Q} \circ + 1_{K}^t Lx_{m} - \frac{q_m}{Q} \circ * a \leq \frac{C_m}{Q} \circ \qquad \forall m \\
\end{eqnarray}

In [19]:


def aux2(cost,q_rep,qa_rep,C_rep,A,I_rep,T):
    M,K = len(q_rep[0]),len(cost)
    
    
    Y = cp.Variable((T,K),boolean=True)
    X = {}
    constraints = []
    sum_X = np.zeros((T,K))
    
    for m in range(M):
        Km=len(A[m])
        
        X[m] = cp.Variable((T,Km),nonneg=True)
        sum_X = sum_X + X[m]@A[m]
        
        I = cp.cumsum(cp.sum(X[m],axis=1)) + I_rep[:,m] - qa_rep[:,m]

        constraints.append(  I <= C_rep[:,m] )
        constraints.append(  I >= q_rep[:,m])
    constraints.append(sum_X <=Y )
    
   
        
    
    
    problem = cp.Problem(cp.Minimize(cp.sum(Y@cost)), constraints)
    problem.solve()
    
    x = np.array(   [   X[m].value.dot(A[m]) for m in range(M)   ]  ) # dimension M x T x K
    x = np.swapaxes(x,0,1)  # dimension T x M x K
    x = np.swapaxes(x,1,2)  # dimension T x K x M 
    y = np.array(Y.value,dtype=bool)
    
    return x, y

def solver2(cost,q,C,r,I_init,Q,T,H=None):
    if H is None : h = T
    else : h = H
        
    M,K = len(q),len(cost)
    q_rep = np.repeat((q/Q)[np.newaxis,:],h,0)
    I_rep = np.repeat((I_init/Q)[np.newaxis,:],h,0)
    qa_rep= np.diag(np.arange(h)).dot(q_rep)
    C_rep = np.repeat((C/Q)[np.newaxis,:],h,0)
    A   = build_A(r)
    
    R_quantity = np.ones(K)
    R_nbr = np.ones(K)*K
    R_selected = np.array([True]*K)
        
    X_weekly, Y_weekly = [],[]
    t = 0
    I = I_rep + 0.
    
    clusters = build_clusters(r)
 
    while t<T : 
        Xt,Yt = np.zeros((h,K,M)), np.zeros((h,K),dtype=bool)

        for (cluster_1r,cluster_s) in clusters :
            
            cluster_r = R_selected & cluster_1r
            
            X1  =  np.zeros((h,K,sum(cluster_s)))
            X1[:,cluster_r,:],Yt[:,cluster_r] = aux2(cost[cluster_r],
                                               q_rep[:,cluster_s],
                                               qa_rep[:,cluster_s],
                                               C_rep[:,cluster_s],
                                               [A[m][:,cluster_r] for m in range(M) if cluster_s[m]],
                                               I[:,cluster_s],
                                               h)

            Xt[:,:,cluster_s] = X1

        X_weekly.append(Xt),Y_weekly.append(Yt)
        t = t + h
        I = I - h*q_rep + Xt.sum(axis=0).sum(axis=0)
        
        R_done = Yt.sum(axis=0)
        R_quantity = R_quantity + R_done
        R_nbr[R_selected] = R_nbr[R_selected] + R_done.sum()
        R_selected = (K*R_quantity >= R_nbr*0.2)
        #print(R_selected)
        


    return Q*np.concatenate(X_weekly,axis=0), np.concatenate(Y_weekly,axis=0)
                        
    
def build_cluster(r):
    K,M = r.shape
    total = 0
    cluster_s = r[0,:]
    cluster_r = np.array([False]*K)
    cluster_r[0] = True

    new_total = sum(cluster_s)+ sum(cluster_r)
    
    while (new_total > total) : 
        total = new_total
        
        for m in range(M):
            if cluster_s[m] : cluster_r = cluster_r | r[:,m]
                
        for k in range(K):
            if cluster_r[k] : cluster_s = cluster_s | r[k,:]
        
        new_total = sum(cluster_s)+ sum(cluster_r)
        
    
    
    return (cluster_r,cluster_s)
    
    
def build_clusters(r):
    K,M = r.shape
    clusters= []
    set_r, set_s = np.array([True]*K), np.array([True]*M)
    
    while sum(set_s) > 0 : 
        
        cluster_little  = build_cluster(r[set_r,:][:,set_s])
        
        cluster = (np.array([False]*K),np.array([False]*M))
        cluster[0][set_r]= cluster_little[0]
        cluster[1][set_s]= cluster_little[1]
        
        clusters.append(cluster)
                
        set_r = set_r & ~cluster[0]
        set_s = set_s & ~cluster[1]
        
        
        
    
    return clusters
    
    
def build_A(B):
    # for each colomn b of B :
    # auxiliary function to transform a boolean vector b to a boolean matrix A 
    # such that each colomn of A represent a True component of R.
    n = len(B)
    L2 = []
    for j in range(len(B[0])):
        b= B[:,j]
        L = []
        for i in range(n) : 
            if b[i] : 
                row = np.zeros(n,dtype=bool)
                row[i]=True
                L.append(row)

        L2.append(np.array(L))
    return L2
        


# Example 1 

## Set data

In [20]:
w1 = Warehouse(position=[0.,0.],capacity = 100, name='w1')
warehouses = [w1]

positions = [
    [0.,10.],
    [3.,10.],
    [10.,3.],
    [10.,0.]
]
capacities   = [5.,3.,2.,4.]
consumptions = [1.,3.,2.,1.5]
tours = [
    [0],[1],[2],[3],[0,1],[1,2],[2,3]
] 
# distance matrix
d = [
    [  0,100,100,100,100],
    [100,  0, 10,150,160],
    [100, 10,  0,140,150],
    [100,150,140,  0, 10],
    [100,160,150, 10,  0]
]



In [21]:
schools = []
for i in range(len(positions)):
    schools.append(  School(positions[i], capacities[i], consumptions[i] , 's%i'%i )  )

routes = [ Route(w1, [schools[m] for m in tour] )  for tour in tours ]

# transform distances to a dataframe
names = [w1.name]+[s.name for s in schools]
D = pd.DataFrame(data=d, columns=names, index=names)


MAP1 = Map(
    schools=schools,
    central = [0.,0.],
    warehouses=warehouses,
    possible_routes=routes,
    Q = 5., D=D
    )

MAP2 = deepcopy(MAP1)

## Run

solve the problem during 30 days, by computing every 5days the optimal solution for the next 5 days (H=5 is kind of the maximum here)

In [22]:
#MAP1.run(solver1,T=5)
#go.Figure(MAP1.fig).show()

In [23]:
#MAP2.run(solver2,T=30,H=5)
#fig = go.Figure(MAP2.fig)
#fig.show()

# Example 2

## Set data

In [24]:
w1 = Warehouse(position=[0.,0.],capacity = 10, name='w1')
w2 = Warehouse(position=[-10.,-10.],capacity = 10, name='w2')
warehouses = [w1,w2]

positions = np.array([
    [20.,20.],
    [10.,40.],
    [50.,6.],
    [30.,14.],
    [0.,-40.],
    [-25.,-40.],
    [-40.,-25.],
    [-40.,0.],
])

capacities   = [5.,3.,2.,4.,
               5.,3.,2.,4.,]
consumptions = [1.,3.,2.,1.5,
               1.,3.,2.,1.5,]

n = 4
tours1,tours2 = [],[]
for i in range(n):
    tours1.append([i])
    tours2.append([i+4])
    for j in range(i+1,n):
        tours1.append([i,j])
        tours2.append([i+n,j+n])



In [25]:
schools = []
for i in range(len(positions)):
    schools.append(  School(positions[i], capacities[i], consumptions[i] , 's%i'%i )  )

routes1 = [ Route(w1, [schools[m] for m in tour] )  for tour in tours1 ]
routes2 = [ Route(w2, [schools[m] for m in tour] )  for tour in tours2 ]


MAP1 = Map(
    schools=schools,
    central = [-20.,20.],
    warehouses=warehouses,
    possible_routes=routes1+routes2,
    Q = 5., Q2 = 20
    )
MAP2 = deepcopy(MAP1)

## Run

solve the problem during 30 days, by computing every 5days the optimal solution for the next 5 days (H=5 is kind of the maximum here)

In [26]:
MAP1.run(solver2,T=30,H=4)
fig = go.Figure(MAP1.fig)
offline.plot(fig, filename="example2.html", auto_open = False)
fig.show()

Running time for solver is 56.905852 sec
